In [1]:
from IPython.display import display
import spotipy
import spotipy.util as util

client_id = 'b69a9985fa8842deb0691b2d0e3f0b69'
client_secret = 'd9e9ae2924174c139a5a9ccb303f9f3a'
redirect_uri = 'http://localhost/'

username = '12182784645'

token = util.prompt_for_user_token(username, 'user-library-read', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)
tracks = []

offset = 0
while True:
    track_set = sp.current_user_saved_tracks(limit=50, offset=offset)['items']
    if track_set:
        tracks += track_set
    else:
        break
    offset += 50
print("%d tracks fetched" % len(tracks))

artist_names = {}
artist_tracks = {}
all_artists = set()
for i in range(len(tracks)):
    tracks[i] = tracks[i]['track']
    track_id = tracks[i]['id']
    
    artist_id = tracks[i]['artists'][0]['id']
    
    if artist_id not in artist_tracks:
        artist_tracks[artist_id] = [track_id]
    else:
        artist_tracks[artist_id].append(track_id)
    
    for artist in tracks[i]['artists']:
        all_artists.add(artist['id'])
        artist_names[artist['id']] = artist['name']
print("%d artists fetched" % len(artist_tracks))



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=b69a9985fa8842deb0691b2d0e3f0b69&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=user-library-read in your browser




Enter the URL you were redirected to:  http://localhost/?code=AQDXIot_A0QWROEl41s92TupaWJWip3O-iVg3SNljskTSoY3Be8SP3Z_4eX8BX-oEQFKSbv1YAmEhl4kegk6acL9QgPQTf0-IOjp_KYuqBP0qbaSeqwXFR88yVDMym5eHvAQmrLQYhiy0JnOLVX0nkNab1pwxtF6OCVxwCqdII58BBYTF_wegn9J4c3UkO5uz9YOJvl6xGr7UKug




725 tracks fetched
354 artists fetched


In [2]:
import networkx as nx

def chunks(n, x):
    return [x[i:i + n] for i in range(0, len(x), n)]

artist_genres = {}
for chunk in chunks(50, list(artist_tracks.keys())):
    for artist in sp.artists(chunk)['artists']:
        id = artist['id']
        artist_genres[id] = artist['genres']
print("Genres fetched")

try:
    G = nx.read_graphml('related_artists.graphml')
except FileNotFoundError:
    G = nx.Graph()
    
i = 0
not_found = 0
for artist_id in all_artists:
    if not G.has_node(artist_id):
        not_found += 1
        related_list = sp.artist_related_artists(artist_id)['artists']

        for related in related_list:
            artist_names[related['id']] = artist['name']
            G.add_edge(artist_id, related['id'])

            sub_related_list = sp.artist_related_artists(related['id'])['artists']
            for sub_related in sub_related_list:
                artist_names[sub_related['id']] = artist['name']
                G.add_edge(related['id'], sub_related['id'])
                i += 1

    #             ssub_related_list = sp.artist_related_artists(sub_related['id'])['artists']
    #             for ssub_related in ssub_related_list:
    #                 artist_names[ssub_related['id']] = artist['name']
    #                 G.add_edge(sub_related['id'], ssub_related['id'])

    if i % 100 ==0:
        print("%d/%d artists processed, %d fetched" % (i, len(all_artists), not_found))
        
for artist in list(artist_tracks.keys()):
    if not G.has_node(artist):
        del artist_tracks[artist]
        
nx.write_graphml(G, 'related_artists.graphml')
print("All artists processed")

Genres fetched
0/635 artists processed, 0 fetched
0/635 artists processed, 0 fetched
0/635 artists processed, 0 fetched
0/635 artists processed, 0 fetched
0/635 artists processed, 0 fetched
0/635 artists processed, 0 fetched
0/635 artists processed, 0 fetched
0/635 artists processed, 0 fetched
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
All artists processed


In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from multiprocessing import Pool
from functools import partial

print("Graph nodes:", nx.number_of_nodes(G))
print("Graph edges:", nx.number_of_edges(G))
print("Graph components:", nx.number_connected_components(G))

artists = list(artist_tracks.keys())
distance = np.zeros((len(artists), len(artists)))
start = datetime.now()
iters = 0

with Pool(4) as pool:
    paths = pool.map(partial(nx.single_source_shortest_path_length, G), artists)

i = 0
for lengths in paths:
    source = artists[i]
    for j in range(len(artists)):
        target = artists[j]
        if target in lengths:
            distance[i][j] = lengths[target]
        else:
            distance[i][j] = np.inf
    iters += 1
    i += 1
    
print(datetime.now() - start)

Graph nodes: 74274
Graph edges: 293294
Graph components: 9
0:01:37.602059


In [4]:
import sklearn.cluster
import sklearn.metrics

connectivity = nx.to_numpy_matrix(G, nodelist=artists)
similarity = np.exp(-distance / distance[ np.isfinite(distance) ].std())
# for n_clusters in range(2, 13):
# dist = np.copy(distance)
# max_value = dist[ np.isfinite(dist) ].max()
# print(max_value)
# for i in range(len(dist)):
#     for j in range(len(dist)):
#         if dist[i][j] == np.inf:
#             dist[i][j] = max_value * 2

#     clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage='average', compute_full_tree=True).fit_predict(dist)
# clustering = sklearn.cluster.DBSCAN(eps=3, min_samples=1, metric='precomputed', n_jobs=-1).fit_predict(dist)
#     print(np.unique(clustering))
#     print("%d cluster silhouette:" % len(np.unique(clustering)), sklearn.metrics.silhouette_score(dist, clustering, metric='precomputed'))
# print(sklearn.metrics.silhouette_score(dist, clustering, metric='precomputed'))

clustering = sklearn.cluster.SpectralClustering(n_clusters=8, affinity='precomputed').fit_predict(similarity)
# clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=8, affinity='precomputed', linkage='average', compute_full_tree=True).fit_predict(dist)
print(np.unique(clustering))
artist_cluster = {}
cluster_artists = {}
for i in range(len(artists)):
    artist_cluster[artists[i]] = clustering[i]
    if clustering[i] not in cluster_artists:
        cluster_artists[clustering[i]] = [artists[i]]
    else:
        cluster_artists[clustering[i]].append(artists[i])
for clus in cluster_artists:
    print(clus, len(cluster_artists[clus]))

[0 1 2 3 4 5 6 7]
7 57
2 57
6 27
1 110
3 27
5 28
0 32
4 15


In [6]:
cd = {}
for i in range(len(clustering)):
    cluster = clustering[i]
    if cluster not in cd:
        cd[cluster] = distance[i]
    else:
        cd[cluster] = np.append(cd[cluster], distance[i])

for cluster in cd:
    print(cluster, cd[cluster].max())

7 9.0
2 8.0
6 8.0
1 9.0
3 8.0
5 8.0
0 8.0
4 7.0


In [32]:
print(artist_cluster)

{}


In [7]:
counts = {'cluster': [], 'genre': [], 'count': []}
for cluster in cluster_artists:
    print('cluster')
    genre_count = dict()
    total_tracks = 0
    for artist in cluster_artists[cluster]:
        for genre in artist_genres[artist]:
            if genre in genre_count:
                genre_count[genre] += len(artist_tracks[artist])
            else:
                genre_count[genre] = len(artist_tracks[artist])
        total_tracks += len(artist_tracks[artist])
    for genre, count in genre_count.items():
        counts['cluster'].append(cluster)
        counts['genre'].append(genre)
        counts['count'].append(count / total_tracks)

counts = pd.DataFrame(counts)
counts.sort_values('count', ascending=False, inplace=True)
original_counts = counts.copy()

clusters_to_name = set(counts['cluster'].unique())
playlists = dict()
while clusters_to_name:
    cluster = counts.iloc[0]['cluster']
    name = counts.iloc[0]['genre']
    playlists[name] = cluster
    counts = counts[ (counts['cluster']!=cluster) & (counts['genre']!=name) ]
    clusters_to_name.remove(cluster)
    
playlist_tracks = dict()
from collections import OrderedDict
for name in playlists:
    playlist_tracks[name] = []
    cluster = playlists[name]
    for artist in cluster_artists[cluster]:
        playlist_tracks[name] += artist_tracks[artist]

cluster
cluster
cluster
cluster
cluster
cluster
cluster
cluster


In [12]:
print(playlist_tracks)

{'hip hop': ['2KBo6O5rkNdtYT3wYjkEkq', '4pdLZsxq0y5oJDb6Cxlokw', '0WFS2dD69nOF7UQOlC0uyE', '5sesHCQziHg85y8dlnNWJC', '2MAaiV0bbeq4FpSwPe2rR8', '1qju0WsEHiDZoOBBDp8aAk', '225WLJ4RlbSMSAZnjMYEt7', '1ulwVPRTo1H1Ae8LGybH5B', '4k2ix82xyGax7S8u6ODb6B', '0V5zAGdvO780GT3gtdxzLt', '0FEvXUlnFqVhDgc5qtFSft', '6t5KVI0lyngpvFr9jkit9C', '3EyNj8ps0rYIAF4xIpFCjc', '0ng3EsPSuIhLOoYMedRh5s', '3R8CATui5dGU42Ddbc2ixE', '5XK1xClRUXSwCUu4I0E1RV', '2yDqLOQH1uQf3FGhcCmcPF', '4F0bdomwbOcuXfmfXJnjDZ', '6W64rkTWrtxwLVKcVbiKlf', '1BkE3G11yJvFBMIwM2nB3g', '4W3ogZe4TwUzfwgwQErOMf', '7h7d9DhSiOVooDxCICbm6e', '4cTBph2s5eMVQH9mu6LqB3', '2KRKpYhnVJbtBJ08kwNk1q', '08Ar75Td2vFXnKv750IAsC', '18uAu5gCnoGGNYQu8LeAr1', '0shK5iZQppbHPQYiy60xs9', '6bUNEbXT7HovLW6BgPCBsb', '1zb9cdsTMrEEL2Ut8rmTb6', '0I3q5fE6wg7LIfHGngUTnV'], 'rap': ['28FGV3ORH14MYORd7s5dlU', '0evt4UZbdhnHtcAnxkm6A1', '60SdxE8apGAxMiRrpbmLY0', '4YYAKYdNI2PgOItxRVmIIB', '5WvAo7DNuPRmk4APhdPzi8', '4IIUaKqGMElZ3rGtuvYlNc', '3OoZagIW3YK8vjt0ws6NOI', '3DXncPQOG4VBw3Q

In [11]:
token = util.prompt_for_user_token(username, 'playlist-modify-private', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token)

for name in playlist_tracks:
    id = sp.user_playlist_create(username, 'sortify '+name, public=False)['id']
    for chunk in chunks(100, playlist_tracks[name]):
        sp.user_playlist_add_tracks(username, id, chunk)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(playlists)